In [11]:
import requests

def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [12]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [24]:
import re

n = re.compile("^Palo Alto",re.IGNORECASE)
companies = db.companies.find({"offices.city":n},{"offices":1, "name":1})

In [25]:
import pandas as pd

df = pd.DataFrame(list(companies))
df

,_id,name,offices
0,52cdef7c4bab8bd675297d8f,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ..."
1,52cdef7c4bab8bd675297d99,Lala,"[{'description': 'Lala Headquarters', 'address..."
2,52cdef7c4bab8bd675297db1,JotSpot,"[{'description': None, 'address1': '', 'addres..."
3,52cdef7c4bab8bd675297dbc,Skype,"[{'description': 'Global HQ', 'address1': '23-..."
4,52cdef7c4bab8bd675297dc8,Topix,"[{'description': None, 'address1': None, 'addr..."
...,...,...,...
194,52cdef7f4bab8bd67529c44e,Tidal Software,"[{'description': '', 'address1': '', 'address2..."
195,52cdef7f4bab8bd67529c454,Bling Nation,"[{'description': 'Headquarters', 'address1': '..."
196,52cdef7f4bab8bd67529c492,AppStream,"[{'description': 'HQ', 'address1': '2300 Geng ..."
197,52cdef7f4bab8bd67529c5ee,Translucent,"[{'description': '', 'address1': '952 Commerci..."


In [26]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
2,None,,,,Palo Alto,CA,USA,37.437328,-122.159928
3,Global HQ,23-29 Rives de Clausen,,L-2165,Luxembourg City,None,LUX,51.518132,-0.131916
3,US office,3210 Porter Drive,,94304,Palo Alto,CA,USA,NaN,NaN
...,...,...,...,...,...,...,...,...,...
194,,,,,Palo Alto,CA,USA,37.437328,-122.159928
195,Headquarters,364 University Ave,,94301,Palo Alto,CA,USA,37.446411,-122.160540
196,HQ,2300 Geng Road,Suite 100,94303,Palo Alto,CA,USA,NaN,NaN
197,,952 Commercial Street,,94303,Palo Alto,CA,USA,37.423822,-122.099602
